# Interactive charts with `pyecharts` library
*note: download the file to view the charts nested in the notebook. Below is the final result.

![title](pyecharts_result.png)

In [1]:
%%capture
import os, sys, re
!{sys.executable} -m pip install pyecharts pyecharts-jupyter-installer

In [2]:
import pandas as pd
import datetime
import calendar

# load echarts lib
import pyecharts.options as opts
from pyecharts.charts import Pie, Bar, Line, Grid, Page
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode

In [3]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

## 1. Load & preprocess data

In [4]:
df_orders = pd.read_csv('data/Orders.csv')
df_details = pd.read_csv('data/Details.csv')
df = df_details.merge(df_orders, on=['Order ID'], how='inner')
df

,Order ID,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode,Order Date,CustomerName,State,City
0,B-25681,1096,658,7,Electronics,Electronic Games,COD,04-06-2018,Bhawna,Madhya Pradesh,Indore
1,B-26055,5729,64,14,Furniture,Chairs,EMI,10-03-2018,Harivansh,Uttar Pradesh,Mathura
2,B-25955,2927,146,8,Furniture,Bookcases,EMI,16-01-2018,Shiva,Maharashtra,Pune
3,B-26093,2847,712,8,Electronics,Printers,Credit Card,27-03-2018,Sarita,Maharashtra,Pune
4,B-25602,2617,1151,4,Electronics,Phones,Credit Card,01-04-2018,Vrinda,Maharashtra,Pune
...,...,...,...,...,...,...,...,...,...,...,...
1495,B-25700,7,-3,2,Clothing,Hankerchief,COD,25-06-2018,Shubhi,Maharashtra,Mumbai
1496,B-25757,3151,-35,7,Clothing,Trousers,EMI,21-08-2018,Vishakha,Madhya Pradesh,Indore
1497,B-25973,4141,1698,13,Electronics,Printers,COD,24-01-2018,Madan Mohan,Uttar Pradesh,Mathura
1498,B-25698,7,-2,1,Clothing,Hankerchief,COD,23-06-2018,Amisha,Tamil Nadu,Chennai


In [5]:
df['Order Date'] = df['Order Date'].apply(lambda x: datetime.datetime.strptime(x, '%d-%m-%Y'))
df['month'] = df['Order Date'].apply(lambda x: calendar.month_name[x.month])
df['month_order'] = pd.Categorical(df['month'], categories=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"], ordered=True)

In [6]:
#  aggregate data by month 
df_monthly = df.groupby('month_order').agg({'Amount':'sum', 'Profit': 'sum', 'Quantity':'sum', 'month':'first'}).reset_index()
df_monthly['month'] = df_monthly['month'].apply(lambda x: x[:3])
#  aggregate data by state 
df_state = df.groupby('State').agg({'Amount':'sum', 'Profit': 'sum', 'Quantity':'sum'}).sort_values('Amount').reset_index()
#  aggregate data by category 
df_category = df.groupby('Category').agg({'Amount':'sum', 'Profit': 'sum', 'Quantity':'sum'}).sort_values('Profit', ascending=False).reset_index()
# aggregate data by payment mode
df_paymode = df.groupby('PaymentMode').agg(count=('Order ID', 'count')).sort_values('count', ascending=False).reset_index()

/var/folders/w0/fdyf6yl15yv3v5rszdt9ypgr0000gn/T/ipykernel_17573/1796553057.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_monthly = df.groupby('month_order').agg({'Amount':'sum', 'Profit': 'sum', 'Quantity':'sum', 'month':'first'}).reset_index()


## 2. Create dashboard with pyecharts

In [7]:
monthly_rev = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.ESSOS, width = '1500px'))
    .add_xaxis(df_monthly['month'].tolist())
    .add_yaxis("Sales Amount", df_monthly['Amount'].tolist())
    .extend_axis(
        yaxis=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(formatter="{value} units")
        )
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                     itemstyle_opts=opts.ItemStyleOpts(opacity=0.8))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Monthly Total Amount & Quantity"),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="${value}")),
        # xaxis_opts=opts.AxisOpts(type_="category", interval=1)
    )
)

rev_line = Line(init_opts=opts.InitOpts(theme=ThemeType.ESSOS)).add_xaxis(df_monthly['month'].tolist()).add_yaxis("Quantity", df_monthly['Quantity'].tolist(), yaxis_index=1)
bar = Bar().add_xaxis(df_monthly['month'].tolist()).add_yaxis("Sales Amount", df_monthly['Amount'].tolist(), yaxis_index=1)
monthly_rev.overlap(rev_line)

state_rev = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.ESSOS))
    .add_xaxis(df_state['State'].tolist())
    .add_yaxis('', df_state['Amount'].tolist(),)
    .reversal_axis()
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Sales Amount by State", pos_right="0%"),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="${value}")),
        datazoom_opts=[{
                'type': 'slider',
                'orient':'vertical',
                'startValue': 'Maharashtra',
                'endValue': 'Rajasthan',
                'zoomLock': True,
            },{
                'type': 'inside',
                'orient':'vertical',
                'startValue': 'Maharashtra',
                'endValue': 'Rajasthan',
                'zoomOnMouseWheel': False,
                'moveOnMouseMove': True,
                'moveOnMouseWheel': True
            },
            # opts.DataZoomOpts(is_zoom_lock=True, pos_bottom=-100),
            ]
    )
)

color_function = """
        function (params) {
            if (params.value < 0 ) {
                return '#ffb248';
            } return '#893448';
        }
        """

monthly_profit = (
    Bar()
    .add_xaxis(df_monthly['month'].tolist())
    .add_yaxis("", df_monthly['Profit'].tolist(),
               itemstyle_opts=opts.ItemStyleOpts(color=JsCode(color_function)))
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Monthly Profit"),
        yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="${value}")),
    )
)

cate_profit = (
    Pie()
    .add(
        series_name="Profit",
        data_pair=[list(z) for z in zip(df_category['Category'], df_category['Profit'])],
        radius=["40%", "70%"],
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="Profit by Category", pos_right="0%"),
                     legend_opts=opts.LegendOpts(pos_right="10%", pos_top="5%"),)
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(
            trigger="item", formatter="{b}: ${c} ({d}%)"
        ),
        label_opts=opts.LabelOpts(formatter="{d}%", position='inside'),
        center= ['82%', '50%']
        # label_opts=opts.LabelOpts(center= ['75%', '50%'])
    )
)

payment_prop = (
    Pie()
    .add(
        series_name="",
        data_pair=[list(z) for z in zip(df_paymode['PaymentMode'], df_paymode['count'])],
        radius="65%",
        label_opts=opts.LabelOpts(is_show=False, position="center"),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="Proportion of Payment Mode"),
                     legend_opts=opts.LegendOpts(pos_left="7%", pos_top="7%"),)
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(
            trigger="item", formatter="{b}: {d}%"
        ),
        label_opts=opts.LabelOpts(formatter="{d}%", position='inside'),
        center= ['20%', '50%']
    )
)

In [8]:
grid1 = (
    Grid(init_opts=opts.InitOpts(theme=ThemeType.ESSOS, width = '1500px'))
        .add(state_rev, grid_opts=opts.GridOpts(pos_left="45%"))
        .add(payment_prop, grid_opts=opts.GridOpts(pos_right="100%"))
)
grid2 = (
    Grid(init_opts=opts.InitOpts(theme=ThemeType.ESSOS, width = '1500px'))
        .add(monthly_profit, grid_opts=opts.GridOpts(pos_right="35%"))
        .add(cate_profit, grid_opts=opts.GridOpts(pos_left="100%"))
)
dashboard = Page(layout=Page.SimplePageLayout)
dashboard.add(monthly_rev, grid1, grid2)
dashboard.render_notebook()
# dashboard.render('pyecharts_dashboard.html')